In [1]:
# Doing all major library imports
import matplotlib.pyplot as plt
import scikitplot as skplt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import re

from sklearn import datasets, metrics
from sklearn.linear_model import LinearRegression, LogisticRegression,LogisticRegressionCV 
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from matplotlib.colors import ListedColormap
from sklearn.pipeline import Pipeline, make_pipeline

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import scikitplot as skplt
from matplotlib.colors import ListedColormap
from sklearn.metrics import classification_report, confusion_matrix

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:.2f}'.format

import wbdata as wb

In [2]:
eq = pd.read_csv ('..\..\support_etc\EQ_11-19-2019 16-33-32-56_timeSeries.csv')
eq.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Product Name,Product Code,Attribute,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,Unnamed: 59
0,Albania,914,Exporter real gross domestic product per capita,Xgdppc,0616. Natural honey,0616,Value,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,3928.00,nan,nan,nan,nan,nan,nan
1,Albania,914,Unit value 95 percent interval - lower bound,uv_LowerBound95,0616. Natural honey,0616,Value,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.31,nan,nan,nan,nan,nan,nan
2,Albania,914,Unit value of exports,uv,"2119. Hides & skins,nes",2119,Value,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.47,0.01,0.01,0.03,0.04,0.03,0.02,0.02,nan
3,Albania,914,Unit value 95 percent interval - lower bound,uv_LowerBound95,"533. Pigments, paints, varnishes & related mat...",533,Value,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.42,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.49,nan,nan,0.46,nan,nan,nan,nan,nan,nan,0.32,nan,0.38,0.45,0.41,0.37,0.39,0.34,0.33,0.34,0.43,nan,0.39,0.42,nan
4,Albania,914,Average unit values imported,uv_Mmean,2512. Mechanical wood pulp,2512,Value,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [3]:
print (eq.shape)

(1200468, 60)


In [4]:
cc = {
'Aruba':'ABW',
'Afghanistan':'AFG',
'Africa':'AFR',
'Angola':'AGO',
'Albania':'ALB',
'Andorra':'AND',
'Andean Region':'ANR',
'Arab World':'ARB',
'United Arab Emirates':'ARE',
'Argentina':'ARG',
'Armenia':'ARM',
'American Samoa':'ASM',
'Antigua and Barbuda':'ATG',
'Australia':'AUS',
'Austria':'AUT',
'Azerbaijan':'AZE',
'Burundi':'BDI',
'East Asia & Pacific (IBRD-only countries)':'BEA',
'Europe & Central Asia (IBRD-only countries)':'BEC',
'Belgium':'BEL',
'Benin':'BEN',
'Burkina Faso':'BFA',
'Bangladesh':'BGD',
'Bulgaria':'BGR',
'IBRD countries classified as high income':'BHI',
'Bahrain':'BHR',
'Bahamas, The':'BHS',
'Bosnia and Herzegovina':'BIH',
'Latin America & the Caribbean (IBRD-only countries)':'BLA',
'Belarus':'BLR',
'Belize':'BLZ',
'Middle East & North Africa (IBRD-only countries)':'BMN',
'Bermuda':'BMU',
'Bolivia':'BOL',
'Brazil':'BRA',
'Barbados':'BRB',
'Brunei Darussalam':'BRN',
'Sub-Saharan Africa (IBRD-only countries)':'BSS',
'Bhutan':'BTN',
'Botswana':'BWA',
'Sub-Saharan Africa (IFC classification)':'CAA',
'Central African Republic':'CAF',
'Canada':'CAN',
'East Asia and the Pacific (IFC classification)':'CEA',
'Central Europe and the Baltics':'CEB',
'Europe and Central Asia (IFC classification)':'CEU',
'Switzerland':'CHE',
'Channel Islands':'CHI',
'Chile':'CHL',
'China':'CHN',
'Cote d\'Ivoire':'CIV',
'Latin America and the Caribbean (IFC classification)':'CLA',
'Middle East and North Africa (IFC classification)':'CME',
'Cameroon':'CMR',
'Congo, Dem. Rep.':'COD',
'Congo, Rep.':'COG',
'Colombia':'COL',
'Comoros':'COM',
'Cabo Verde':'CPV',
'Costa Rica':'CRI',
'South Asia (IFC classification)':'CSA',
'Caribbean small states':'CSS',
'Cuba':'CUB',
'Curacao':'CUW',
'Cayman Islands':'CYM',
'Cyprus':'CYP',
'Czech Republic':'CZE',
'East Asia & Pacific (IDA-eligible countries)':'DEA',
'Europe & Central Asia (IDA-eligible countries)':'DEC',
'Germany':'DEU',
'IDA countries classified as Fragile Situations':'DFS',
'Djibouti':'DJI',
'Latin America & the Caribbean (IDA-eligible countries)':'DLA',
'Dominica':'DMA',
'Middle East & North Africa (IDA-eligible countries)':'DMN',
'IDA countries not classified as Fragile Situations':'DNF',
'Denmark':'DNK',
'IDA countries in Sub-Saharan Africa not classified as fragile situations ':'DNS',
'Dominican Republic':'DOM',
'South Asia (IDA-eligible countries)':'DSA',
'IDA countries in Sub-Saharan Africa classified as fragile situations ':'DSF',
'Sub-Saharan Africa (IDA-eligible countries)':'DSS',
'IDA total, excluding Sub-Saharan Africa':'DXS',
'Algeria':'DZA',
'East Asia & Pacific (excluding high income)':'EAP',
'Early-demographic dividend':'EAR',
'East Asia & Pacific':'EAS',
'Europe & Central Asia (excluding high income)':'ECA',
'Europe & Central Asia':'ECS',
'Ecuador':'ECU',
'Egypt, Arab Rep.':'EGY',
'Euro area':'EMU',
'Eritrea':'ERI',
'Spain':'ESP',
'Estonia':'EST',
'Ethiopia':'ETH',
'European Union':'EUU',
'Fragile and conflict affected situations':'FCS',
'Finland':'FIN',
'Fiji':'FJI',
'France':'FRA',
'Faroe Islands':'FRO',
'Micronesia, Fed. Sts.':'FSM',
'IDA countries classified as fragile situations, excluding Sub-Saharan Africa':'FXS',
'Gabon':'GAB',
'United Kingdom':'GBR',
'Georgia':'GEO',
'Ghana':'GHA',
'Gibraltar':'GIB',
'Guinea':'GIN',
'Gambia, The':'GMB',
'Guinea-Bissau':'GNB',
'Equatorial Guinea':'GNQ',
'Greece':'GRC',
'Grenada':'GRD',
'Greenland':'GRL',
'Guatemala':'GTM',
'Guam':'GUM',
'Guyana':'GUY',
'High income':'HIC',
'Hong Kong SAR, China':'HKG',
'Honduras':'HND',
'Heavily indebted poor countries (HIPC)':'HPC',
'Croatia':'HRV',
'Haiti':'HTI',
'Hungary':'HUN',
'IBRD, including blend':'IBB',
'IBRD only':'IBD',
'IDA & IBRD total':'IBT',
'IDA total':'IDA',
'IDA blend':'IDB',
'Indonesia':'IDN',
'IDA only':'IDX',
'Isle of Man':'IMN',
'India':'IND',
'Not classified':'INX',
'Ireland':'IRL',
'Iran, Islamic Rep.':'IRN',
'Iraq':'IRQ',
'Iceland':'ISL',
'Israel':'ISR',
'Italy':'ITA',
'Jamaica':'JAM',
'Jordan':'JOR',
'Japan':'JPN',
'Kazakhstan':'KAZ',
'Kenya':'KEN',
'Kyrgyz Republic':'KGZ',
'Cambodia':'KHM',
'Kiribati':'KIR',
'St. Kitts and Nevis':'KNA',
'Korea, Rep.':'KOR',
'Kuwait':'KWT',
'Latin America & Caribbean (excluding high income)':'LAC',
'Lao PDR':'LAO',
'Lebanon':'LBN',
'Liberia':'LBR',
'Libya':'LBY',
'St. Lucia':'LCA',
'Latin America & Caribbean ':'LCN',
'Latin America and the Caribbean':'LCR',
'Least developed countries: UN classification':'LDC',
'Low income':'LIC',
'Liechtenstein':'LIE',
'Sri Lanka':'LKA',
'Lower middle income':'LMC',
'Low & middle income':'LMY',
'Lesotho':'LSO',
'Late-demographic dividend':'LTE',
'Lithuania':'LTU',
'Luxembourg':'LUX',
'Latvia':'LVA',
'Macao SAR, China':'MAC',
'St. Martin (French part)':'MAF',
'Morocco':'MAR',
'Central America':'MCA',
'Monaco':'MCO',
'Moldova':'MDA',
'Middle East (developing only)':'MDE',
'Madagascar':'MDG',
'Maldives':'MDV',
'Middle East & North Africa':'MEA',
'Mexico':'MEX',
'Marshall Islands':'MHL',
'Middle income':'MIC',
'North Macedonia':'MKD',
'Mali':'MLI',
'Malta':'MLT',
'Myanmar':'MMR',
'Middle East & North Africa (excluding high income)':'MNA',
'Montenegro':'MNE',
'Mongolia':'MNG',
'Northern Mariana Islands':'MNP',
'Mozambique':'MOZ',
'Mauritania':'MRT',
'Mauritius':'MUS',
'Malawi':'MWI',
'Malaysia':'MYS',
'North America':'NAC',
'North Africa':'NAF',
'Namibia':'NAM',
'New Caledonia':'NCL',
'Niger':'NER',
'Nigeria':'NGA',
'Nicaragua':'NIC',
'Netherlands':'NLD',
'Non-resource rich Sub-Saharan Africa countries, of which landlocked':'NLS',
'Norway':'NOR',
'Nepal':'NPL',
'Non-resource rich Sub-Saharan Africa countries':'NRS',
'Nauru':'NRU',
'IDA countries not classified as fragile situations, excluding Sub-Saharan Africa':'NXS',
'New Zealand':'NZL',
'OECD members':'OED',
'Oman':'OMN',
'Other small states':'OSS',
'Pakistan':'PAK',
'Panama':'PAN',
'Peru':'PER',
'Philippines':'PHL',
'Palau':'PLW',
'Papua New Guinea':'PNG',
'Poland':'POL',
'Pre-demographic dividend':'PRE',
'Puerto Rico':'PRI',
'Korea, Dem. People???s Rep.':'PRK',
'Portugal':'PRT',
'Paraguay':'PRY',
'West Bank and Gaza':'PSE',
'Pacific island small states':'PSS',
'Post-demographic dividend':'PST',
'French Polynesia':'PYF',
'Qatar':'QAT',
'Romania':'ROU',
'Resource rich Sub-Saharan Africa countries':'RRS',
'Resource rich Sub-Saharan Africa countries, of which oil exporters':'RSO',
'Russian Federation':'RUS',
'Rwanda':'RWA',
'South Asia':'SAS',
'Saudi Arabia':'SAU',
'Southern Cone':'SCE',
'Sudan':'SDN',
'Senegal':'SEN',
'Singapore':'SGP',
'Solomon Islands':'SLB',
'Sierra Leone':'SLE',
'El Salvador':'SLV',
'San Marino':'SMR',
'Somalia':'SOM',
'Serbia':'SRB',
'Sub-Saharan Africa (excluding high income)':'SSA',
'South Sudan':'SSD',
'Sub-Saharan Africa ':'SSF',
'Small states':'SST',
'Sao Tome and Principe':'STP',
'Suriname':'SUR',
'Slovak Republic':'SVK',
'Slovenia':'SVN',
'Sweden':'SWE',
'Eswatini':'SWZ',
'Sint Maarten (Dutch part)':'SXM',
'Sub-Saharan Africa excluding South Africa':'SXZ',
'Seychelles':'SYC',
'Syrian Arab Republic':'SYR',
'Turks and Caicos Islands':'TCA',
'Chad':'TCD',
'East Asia & Pacific (IDA & IBRD countries)':'TEA',
'Europe & Central Asia (IDA & IBRD countries)':'TEC',
'Togo':'TGO',
'Thailand':'THA',
'Tajikistan':'TJK',
'Turkmenistan':'TKM',
'Latin America & the Caribbean (IDA & IBRD countries)':'TLA',
'Timor-Leste':'TLS',
'Middle East & North Africa (IDA & IBRD countries)':'TMN',
'Tonga':'TON',
'South Asia (IDA & IBRD)':'TSA',
'Sub-Saharan Africa (IDA & IBRD countries)':'TSS',
'Trinidad and Tobago':'TTO',
'Tunisia':'TUN',
'Turkey':'TUR',
'Tuvalu':'TUV',
'Taiwan, China':'TWN',
'Tanzania':'TZA',
'Uganda':'UGA',
'Ukraine':'UKR',
'Upper middle income':'UMC',
'Uruguay':'URY',
'United States':'USA',
'Uzbekistan':'UZB',
'St. Vincent and the Grenadines':'VCT',
'Venezuela, RB':'VEN',
'British Virgin Islands':'VGB',
'Virgin Islands (U.S.)':'VIR',
'Vietnam':'VNM',
'Vanuatu':'VUT',
'World':'WLD',
'Samoa':'WSM',
'Kosovo':'XKX',
'Sub-Saharan Africa excluding South Africa and Nigeria':'XZN',
'Yemen, Rep.':'YEM',
'South Africa':'ZAF',
'Zambia':'ZMB',
'Zimbabwe':'ZWE',
}

In [5]:
eq['country_code'] = eq['Country Name'].replace(cc)

In [6]:
eq.isnull().sum()

Country Name            0
Country Code            0
Indicator Name          0
Indicator Code          0
Product Name            0
Product Code            0
Attribute               0
1963               924886
1964               891618
1965               864366
1966               851363
1967               838848
1968               833133
1969               825962
1970               762554
1971               744717
1972               733441
1973               717901
1974               703896
1975               699502
1976               694050
1977               681512
1978               683274
1979               675517
1980               667729
1981               670752
1982               681780
1983               686595
1984               692370
1985               686872
1986               672753
1987               668696
1988               666654
1989               664813
1990               645265
1991               648244
1992               639344
1993               581349
1994        

In [7]:
list(eq.columns)

['Country Name',
 'Country Code',
 'Indicator Name',
 'Indicator Code',
 'Product Name',
 'Product Code',
 'Attribute',
 '1963',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 'Unnamed: 59',
 'country_code']

In [8]:
eq.dtypes

Country Name       object
Country Code        int64
Indicator Name     object
Indicator Code     object
Product Name       object
Product Code       object
Attribute          object
1963              float64
1964              float64
1965              float64
1966              float64
1967              float64
1968              float64
1969              float64
1970              float64
1971              float64
1972              float64
1973              float64
1974              float64
1975              float64
1976              float64
1977              float64
1978              float64
1979              float64
1980              float64
1981              float64
1982              float64
1983              float64
1984              float64
1985              float64
1986              float64
1987              float64
1988              float64
1989              float64
1990              float64
1991              float64
1992              float64
1993              float64
1994        

In [30]:
eq.drop(columns = [
     '1963',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
     'Country Code',
 'Indicator Code',
 'Product Code',
    'Attribute',
    'Unnamed: 59'
], inplace=True)

In [31]:
eq.head()

,Country Name,Indicator Name,Product Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,country_code
0,Albania,Exporter real gross domestic product per capita,0616. Natural honey,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,3928.00,nan,nan,nan,nan,nan,ALB
1,Albania,Unit value 95 percent interval - lower bound,0616. Natural honey,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.31,nan,nan,nan,nan,nan,ALB
2,Albania,Unit value of exports,"2119. Hides & skins,nes",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.47,0.01,0.01,0.03,0.04,0.03,0.02,0.02,ALB
3,Albania,Unit value 95 percent interval - lower bound,"533. Pigments, paints, varnishes & related mat...",nan,0.49,nan,nan,0.46,nan,nan,nan,nan,nan,nan,0.32,nan,0.38,0.45,0.41,0.37,0.39,0.34,0.33,0.34,0.43,nan,0.39,0.42,ALB
4,Albania,Average unit values imported,2512. Mechanical wood pulp,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,ALB


In [33]:
eq_melt = pd.melt(eq, id_vars=['Country Name', 'country_code','Indicator Name', 'Product Name'])
eq_melt.head()

,Country Name,country_code,Indicator Name,Product Name,variable,value
0,Albania,ALB,Exporter real gross domestic product per capita,0616. Natural honey,1990,nan
1,Albania,ALB,Unit value 95 percent interval - lower bound,0616. Natural honey,1990,nan
2,Albania,ALB,Unit value of exports,"2119. Hides & skins,nes",1990,nan
3,Albania,ALB,Unit value 95 percent interval - lower bound,"533. Pigments, paints, varnishes & related mat...",1990,nan
4,Albania,ALB,Average unit values imported,2512. Mechanical wood pulp,1990,nan


In [34]:
eq_ah = pd.pivot_table (eq_melt, index =['Country Name', 'country_code','variable'], columns = 'Indicator Name', values = 'value', aggfunc=np.sum) 
eq_ah.head()

Indicator Name                      Average quality imported  \
Country Name country_code variable                             
Albania      ALB          1990                        209.76   
                          1991                        202.47   
                          1992                        208.54   
                          1993                        218.84   
                          1994                        259.96   

Indicator Name                      Average unit values imported  \
Country Name country_code variable                                 
Albania      ALB          1990                            171.91   
                          1991                            173.35   
                          1992                            178.87   
                          1993                            180.01   
                          1994                            211.51   

Indicator Name                      Export Quality Index  \
Country Name country_code variable                         
Albania      ALB          1990                    180.82   
                          1991                    171.33   
                          1992                    172.58   
                          1993                    188.09   
                          1994                    223.96   

Indicator Name                      Export quality 95 percent interval - lower bound  \
Country Name country_code variable                                                     
Albania      ALB          1990                                                149.50   
                          1991                                                147.78   
                          1992                                                150.31   
                          1993                                                163.30   
                          1994                                                195.58   

Indicator Name                      Export quality 95 percent interval - upper bound  \
Country Name country_code variable                                                     
Albania      ALB          1990                                                234.16   
                          1991                                                225.89   
                          1992                                                232.87   
                          1993                                                244.42   
                          1994                                                285.25   

Indicator Name                      Exporter real gross domestic product per capita  \
Country Name country_code variable                                                    
Albania      ALB          1990                                            417138.00   
                          1991                                            283503.00   
                          1992                                            273460.00   
                          1993                                            317840.00   
                          1994                                            406096.00   

Indicator Name                      Trade value of exports  \
Country Name country_code variable                           
Albania      ALB          1990                  1117031.15   
                          1991                   706857.34   
                          1992                   559187.80   
                          1993                   702183.42   
                          1994                  1056304.80   

Indicator Name                      Unit value 95 percent interval - lower bound  \
Country Name country_code variable                                                 
Albania      ALB          1990                                             66.61   
                          1991                                             64.45   
                          1992        

In [35]:
eq_ah.reset_index(inplace=True)

In [36]:
eq_ah.head()

Indicator Name,Country Name,country_code,variable,Average quality imported,Average unit values imported,Export Quality Index,Export quality 95 percent interval - lower bound,Export quality 95 percent interval - upper bound,Exporter real gross domestic product per capita,Trade value of exports,Unit value 95 percent interval - lower bound,Unit value 95 percent interval - upper bound,Unit value of exports
0,Albania,ALB,1990,209.76,171.91,180.82,149.50,234.16,417138.00,1117031.15,66.61,245.48,108.60
1,Albania,ALB,1991,202.47,173.35,171.33,147.78,225.89,283503.00,706857.34,64.45,214.00,100.43
2,Albania,ALB,1992,208.54,178.87,172.58,150.31,232.87,273460.00,559187.80,65.48,232.55,96.29
3,Albania,ALB,1993,218.84,180.01,188.09,163.30,244.42,317840.00,702183.42,67.65,239.73,105.18
4,Albania,ALB,1994,259.96,211.51,223.96,195.58,285.25,406096.00,1056304.80,75.03,296.36,153.75


In [37]:
eq_ah.rename (columns = {
    'variable': 'date',
}, inplace=True)
eq_ah.head()

Indicator Name,Country Name,country_code,date,Average quality imported,Average unit values imported,Export Quality Index,Export quality 95 percent interval - lower bound,Export quality 95 percent interval - upper bound,Exporter real gross domestic product per capita,Trade value of exports,Unit value 95 percent interval - lower bound,Unit value 95 percent interval - upper bound,Unit value of exports
0,Albania,ALB,1990,209.76,171.91,180.82,149.50,234.16,417138.00,1117031.15,66.61,245.48,108.60
1,Albania,ALB,1991,202.47,173.35,171.33,147.78,225.89,283503.00,706857.34,64.45,214.00,100.43
2,Albania,ALB,1992,208.54,178.87,172.58,150.31,232.87,273460.00,559187.80,65.48,232.55,96.29
3,Albania,ALB,1993,218.84,180.01,188.09,163.30,244.42,317840.00,702183.42,67.65,239.73,105.18
4,Albania,ALB,1994,259.96,211.51,223.96,195.58,285.25,406096.00,1056304.80,75.03,296.36,153.75


In [38]:
eq_ah.to_csv ('eq_imf_output.csv', index=False)